In [1]:
import uuid
from flask import Flask, request, jsonify
from utilities import logger
from flask_cors import CORS
import sys
from xinference.client import Client

mylogger = logger.Logger(name='llmtool', debug=True).logger  
# app = Flask(__name__)  
# CORS(app)

# 使用字典模拟用户存储，实际项目中应使用数据库
users_db = {
    "admin": {"password": "123456"},
    'zz': {'password': 'zz'},
    'xyz': {'password': 'abc'}
}

# 存储登录成功的用户session数据
sessions = {}

x_addr = 'http://10.101.9.50:9998'
mylogger.debug(f"Start to connect xinference server at {x_addr}")
try:
    client = Client(x_addr)    
    model_uid = client.launch_model(model_name="bge-large-zh-v1.5", model_type="embedding")    
    model = client.get_model(model_uid)
except Exception as e:
    mylogger.error(f"Something goes wrong: {e}")   
def create_embedding_bge(sentence):
    result = model.create_embedding(sentence)
    embedding = result['data'][0]['embedding']
    # LOG.debug(f"length of embedding {len(embedding)}, {embedding[:5]}")
    return embedding

2024-03-22 22:59:27.376 | DEBUG    | __main__:<module>:23 - Start to connect xinference server at http://10.101.9.50:9998


In [2]:
vec = create_embedding_bge('test sentence')

In [6]:
import requests

# 调用生成列表的服务
question = '可以根据需要修改question参数'  # 可以根据需要修改question参数
response = requests.post('http://127.0.0.1:5111/genvec', json={'question': question})
vec = create_embedding_bge(question)
if response.status_code == 200:
    float_list = response.json()
    # 假设 float_list 和 vec 是之前获取的列表
# 检查类型是否都是 list
    if isinstance(float_list, list) and isinstance(vec, list):
        # 检查长度是否相同
        if len(float_list) == len(vec):
            # 逐个比较元素是否一致
            if all(a == b for a, b in zip(float_list, vec)):
                mylogger.debug("float_list 和 vec 相同")
            else:
                mylogger.error("float_list 和 vec 不同")
        else:
            mylogger.error("float_list 和 vec 长度不同")
    else:
        mylogger.error("float_list 和 vec 不是列表类型")
        # 判断列表长度
        mylogger.debug(f"Length of the list: {len(float_list)}")    
    
else:
    mylogger.error("Failed to retrieve the list.")

2024-03-22 23:08:26.596 | DEBUG    | __main__:<module>:10 - Length of the list: 1024
2024-03-22 23:08:26.599 | DEBUG    | __main__:<module>:13 - First ten elements:
2024-03-22 23:08:26.602 | DEBUG    | __main__:<module>:14 - [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]
2024-03-22 23:08:26.604 | DEBUG    | __main__:<module>:17 - Last ten elements:
2024-03-22 23:08:26.605 | DEBUG    | __main__:<module>:18 - [10.15, 10.16, 10.17, 10.18, 10.19, 10.2, 10.21, 10.22, 10.23, 10.24]
